In [6]:
import boto3
import json
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

def load_jsonl_from_s3(bucket_name, file_key):
    """
    Load a JSONL file from an S3 bucket using credentials from environment variables.

    Parameters:
    - bucket_name: str - Name of the S3 bucket.
    - file_key: str - Key (path) of the JSONL file in the bucket.

    Returns:
    - List of Python dictionaries loaded from the JSONL file.
    """
    # Get AWS credentials from environment variables
    aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
    aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")
    region_name = os.getenv("AWS_REGION")
    
    # Initialize an S3 client
    s3_client = boto3.client(
        's3',
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key,
        region_name=region_name
    )
    
    # Retrieve the file content
    response = s3_client.get_object(Bucket=bucket_name, Key=file_key)
    content = response['Body'].read().decode('utf-8')
    
    # Parse the JSONL content
    data = [json.loads(line) for line in content.splitlines() if line.strip()]
    
    return data

# Example usage
if __name__ == "__main__":
    bucket_name = "small-reviews584"
    file_key = "data/reviews_small.jsonl"
    
    data = load_jsonl_from_s3(bucket_name, file_key)